In [1]:
using Pkg
using Revise
Pkg.activate("..") 
Pkg.instantiate()
using JPEC, Plots
gr() 

  Activating project at `~/git/JPEC11/JPEC`
    Updating registry at `~/.julia/registries/General.toml`
    Updating `~/git/JPEC11/JPEC/Project.toml`
  [0c46a032] + DifferentialEquations v7.16.1
  [7a1cc6ca] + FFTW v1.9.0
  [f67ccb44] + HDF5 v0.17.2
  [91a5bcdd] + Plots v1.40.17
  [37e2e46d] ~ LinearAlgebra ⇒ v1.11.0
  [44cfe95a] ~ Pkg ⇒ v1.11.0
  [de0858da] ~ Printf ⇒ v1.11.0
  [fa267f1f] ~ TOML ⇒ v1.0.3
  [8dfed614] ~ Test ⇒ v1.11.0
    Updating `~/git/JPEC11/JPEC/Manifest.toml`
  [47edcb42] + ADTypes v1.16.0
  [621f4979] + AbstractFFTs v1.5.0
  [7d9f7c33] + Accessors v0.1.42
  [79e6a3ab] + Adapt v4.3.0
  [66dad0bd] + AliasTables v1.1.3
  [a95523ee] + AlmostBlockDiagonals v0.1.10
  [ec485272] + ArnoldiMethod v0.4.0
  [4fba245c] + ArrayInterface v7.19.0
  [4c555306] + ArrayLayouts v1.11.2
  [aae01518] + BandedMatrices v1.9.4
  [d1d4a3ce] + BitFlags v0.1.9
  [62783981] + BitTwiddlingConvenienceFunctions v0.1.6
  [764a87c0] + BoundaryValueDiffEq v5.18.0
  [7227322d] + BoundaryValueDiffE

Plots.GRBackend()

In [2]:
using Revise
include("../src/DCON/fourfit.jl")
import .fourfit_make_metric, .MetricData, .fourfit_make_matrix, .MatrixData, .compute_eigenvalues

### Reading Eq

In [16]:
# 1. Define the input parameters for the equilibrium solver.
#    - eq_filename: The name of the g-file we just created.
#    - eq_type: "efit" for a standard g-file.
#    - jac_type: "boozer" or "hamada" for the output coordinates.
#    - mpsi, mtheta: Resolution of the output grid.
    equil_control = JPEC.Equilibrium.EquilibriumControl(;
    eq_filename="beta_1.00",        # eq_filename
    eq_type="efit",          # eq_type
    jac_type="hamada",        # jac_type
    grid_type="ldp",
    psilow=0.01,             # psilow
    psihigh=0.990)             # psihigh
equil_config = JPEC.Equilibrium.EquilibriumConfig(equil_control,JPEC.Equilibrium.EquilibriumOutput())
# 2. Run the main equilibrium setup function.
#    This will read the file, solve the direct problem, and return the final object.
println("Starting equilibrium reconstruction...")
plasma_eq = JPEC.Equilibrium.setup_equilibrium(equil_config)
#plasma_eq = JPEC.Equilibrium.setup_equilibrium("./DIIID_example/equil.toml")
println("Equilibrium reconstruction complete.")

Starting equilibrium reconstruction...
Equilibrium file: beta_1.00
--> Processing EFIT g-file: beta_1.00

┌ Info: Forcing hamada coordinate jacobian exponents: power_*
└ @ JPEC.Equilibrium /Users/seoda-eun/git/JPEC11/JPEC/src/Equilibrium/EquilibriumTypes.jl:48



--> Parsed from header: nw=129, nh=128
--> All main data blocks parsed successfully.
--> Creating 1D profile splines...
--> 1D Spline fitting complete.
--> Creating 2D psi spline...
--> 2D Spline fitting complete.
--- Starting Direct Equilibrium Processing ---
Finding magnetic axis...
  Iter  1: R = 3.115238, Z = 0.000008, |ΔR|=1.15e-01, |ΔZ|=1.11e-05
  Iter  2: R = 3.117635, Z = -0.000003, |ΔR|=2.40e-03, |ΔZ|=1.10e-05
  Iter  3: R = 3.117636, Z = -0.000003, |ΔR|=1.07e-06, |ΔZ|=1.04e-09
  Iter  4: R = 3.117636, Z = -0.000003, |ΔR|=2.12e-13, |ΔZ|=5.62e-16
Magnetic axis found at R=3.1176357129137466, Z=-2.9184696943092166e-6.
Finding inboard separatrix crossing...
  Restart attempt 1/6 with initial R = 2.675802
inboard separatrix found at R=1.5000175239830411.
Finding outboard separatrix crossing...
  Restart attempt 1/6 with initial R = 3.500684
outboard separatrix found at R=4.499995001114418.
Starting loop over flux surfaces...
--> Processing surface ipsi = 128 / 128 (ψ_norm = 0.9900

In [17]:
fieldnames(typeof(plasma_eq.rzphi))

(:handle, :_xs, :_ys, :_fs, :mx, :my, :nqty, :bctypex, :bctypey, :_fsx, :_fsy, :_fsxy)

### CAlc. Metric

In [5]:
include("../src/DCON/fourfit.jl")
import .fourfit_make_metric, .MetricData, .fourfit_make_matrix, .MatrixData, .compute_eigenvalues

In [18]:
# Fourfit Metric Calculation Example
println("📚 Fourfit Metric Calculation")
println("="^30)

using .FourfitMetric
metric_result = nothing
try
    # Metric tensor calculation
    metric_result = FourfitMetric.fourfit_make_metric(
        plasma_eq.rzphi,  # 2D geometry spline
        plasma_eq.sq;     # 1D profile spline
        mpsi=128,          # Radial grid points
        mtheta=256,        # Poloidal grid points  
        mband=10,          # Fourier bandwidth
        fft_flag=true,    # Use FFT
        verbose=true      # Detailed output
    )
    
    println("✅ Metric calculation complete!")
    println("   Grid size: $(size(metric_result.fs))")
 
catch e
    println("❌Error: $e")
end

📚 Fourfit Metric Calculation
🔧 Starting metric tensor calculation...
   Grid: 129 × 257, mband: 10
📊 Computing metric tensor components on grid...
---- DEBUG: ipsi=64.0, itheta=128.0 ----
psi_coord = 0.4999999999999999
theta_coord = 0.49609375
bicube_eval: f = [1.0556705372876192, -0.0033389596882782975, -0.02808040164119698, 186.5043062876643]
bicube_eval: fx = [2.4382568339151183, -0.0033018151911684934, -0.06546817349115953, 107.74688881364901]
bicube_eval: fy = [-0.7733913169930531, 0.852669494134877, 7.186190745690982, 0.0]
rfac = 1.0274582898043205
eta = 3.0960696585289678
r = 2.091236149391108
jac = 186.5043062876643
jac1 = 107.74688881364901
Contravariant basis vectors v:
  v[1,:] = [0.006362039923106128, -0.00011428992988609545, -0.0007340817687509711]
  v[2,:] = [-0.0020179770918524655, 0.06412880622547774, 0.08057734517201763]
  v[3,:] = [0.0, 0.0, 0.07045212257689355]
g11 = 0.007651803601175929
g22 = 1.9786772597603564
g33 = 0.9257144181130735
g23 = 1.0587560384372956
g31 =

In [7]:
include("../src/DCON/fourfit.jl")
import .fourfit_make_metric, .MetricData, .fourfit_make_matrix, .MatrixData, .compute_eigenvalues

### Calc. Matric

In [8]:
using Revise

In [9]:
include("../src/DCON/fourfit.jl")
import .fourfit_make_metric, .MetricData, .fourfit_make_matrix, .MatrixData, .compute_eigenvalues

In [19]:
# Fourfit Matrix Calculation - Complete Example
println("🚀 Fourfit Matrix Calculation")
println("="^40)

using .FourfitMetric
matrix_result = nothing
try
    # Step 1: Matrix calculation with correct arguments
    println("2️⃣  Calculating MHD Coefficient Matrix...")
    
    matrix_result = FourfitMetric.fourfit_make_matrix(
        metric_result,       # metric tensor data
        plasma_eq.sq,        # 1D profile spline
        plasma_eq.rzphi,     # 2D geometry spline
        1.367;               # psio (toroidal flux normalization)
        nn=1,                # toroidal mode number
        mlow=-4,             # minimum poloidal mode
        mhigh=4,             # maximum poloidal mode
        power_flag=false,
        feval_flag=false,
        sas_flag=true,
        verbose=true
    )
    
    # Step 2: Analyze results
    println("3️⃣  Analyzing Results:")
    println("   Matrix storage sizes:")
    println("      F-matrix: $(size(matrix_result.fmats))")
    println("      G-matrix: $(size(matrix_result.gmats))")
    println("      K-matrix: $(size(matrix_result.kmats))")
    println("   Mode range: $(matrix_result.mlow) to $(matrix_result.mhigh)")
    println("   Number of modes: $(matrix_result.mpert)")
    
    println("✅ Matrix calculation completed successfully!")
    
catch e
    println("❌ Error occurred: $e")
    println("🔍 Error type: $(typeof(e))")
end

🚀 Fourfit Matrix Calculation
2️⃣  Calculating MHD Coefficient Matrix...
🚀 Fourfit Matrix Calculation - Complete Implementation
📊 Configuration:
   Mode range: -4 to 4 (total: 9 modes)
   Toroidal mode: n = 1
   Psi surfaces: 0 to 128
   Metric data size: (129, 88)

1️⃣  Allocating matrix storage...
   ✅ Matrix dimensions: 9 × 9
   ✅ Hermitian storage size: 44
   ✅ K-matrix storage size: 189

2️⃣  Computing matrices on each flux surface...
   Progress: 0.0% (ipsi = 0/128, ψ = 0.01)
   Progress: 9.4% (ipsi = 12/128, ψ = 0.0311)
   Progress: 18.8% (ipsi = 24/128, ψ = 0.0926)
   Progress: 28.1% (ipsi = 36/128, ψ = 0.1891)
   Progress: 37.5% (ipsi = 48/128, ψ = 0.3125)
   Progress: 46.9% (ipsi = 60/128, ψ = 0.452)
   Progress: 56.2% (ipsi = 72/128, ψ = 0.5956)
   Progress: 65.6% (ipsi = 84/128, ψ = 0.731)
   Progress: 75.0% (ipsi = 96/128, ψ = 0.8465)
   Progress: 84.4% (ipsi = 108/128, ψ = 0.9321)
   Progress: 93.8% (ipsi = 120/128, ψ = 0.9806)

3️⃣  Setting power factors...

4️⃣  Creating

### Redaing imats(original DCON output)

In [12]:
using XLSX

In [13]:
# ────────────────────────────────────────────────────────────────
# Internal constants - Excel column positions (1-based)
const _COLS = Dict(
    :A => (4, 5),
    :B => (6, 7),
    :C => (8, 9),
    :D => (10, 11),
    :E => (12, 13),
    :H => (14, 15),
)
# ────────────────────────────────────────────────────────────────
mutable struct ImatsReader
    path      :: String
    psi       :: Vector{Float64}
    m_values  :: Vector{Int}
    npsi      :: Int
    mpert     :: Int
    A         :: Array{ComplexF64,3}
    B         :: Array{ComplexF64,3}
    C         :: Array{ComplexF64,3}
    D         :: Array{ComplexF64,3}
    E         :: Array{ComplexF64,3}
    H         :: Array{ComplexF64,3}
end

function ImatsReader(path::AbstractString)
    isfile(path) || throw(ArgumentError("File not found: $path"))
    # Temporary storage for variables
    temp_storage = []
    # 1) Open Excel file and read all data rows temporarily
    XLSX.openxlsx(path) do xf
        sheet = xf[1]
        # Use XLSX.eachtablerow to iterate through rows
        for row in XLSX.eachtablerow(sheet)
            # Skip if first column value is not a number (header or empty row)
            ψ = row[1]
            isa(ψ, Number) || continue
            # Read mi, mj and other values from the row
            mi = Int(row[2])
            mj = Int(row[3])
            # Store entire row data (columns containing real/imaginary parts of A~H)
            # Read data up to 15th column
            row_data = [row[c] for c in 1:15]
            push!(temp_storage, (ψ, mi, mj, row_data))
        end
    end # End of XLSX.openxlsx do block
    isempty(temp_storage) && error("No numeric data rows found in $path")
    
    # 2) Organize ψ / m lists
    psi_vals = sort(unique(getindex.(temp_storage, 1)))
    m_vals_all = vcat(getindex.(temp_storage, 2), getindex.(temp_storage, 3))
    m_vals   = sort(unique(m_vals_all))
    nψ       = length(psi_vals)
    mpert    = length(m_vals)
    
    # Create dictionaries for value -> index conversion
    ψidx = Dict(v => i for (i, v) in enumerate(psi_vals))
    midx = Dict(v => i for (i, v) in enumerate(m_vals))
    
    # 3) Initialize empty arrays to store final matrices
    shp = (nψ, mpert, mpert)
    mats = Dict(name => zeros(ComplexF64, shp) for name in keys(_COLS))
    
    # 4) Fill arrays with temporarily stored data
    for (ψ, mi, mj, row_cells) in temp_storage
        k = ψidx[ψ]
        i = midx[mi]
        j = midx[mj]
        for (name, (recol, imcol)) in _COLS
            # row_cells has 1-based indexing
            real_val = row_cells[recol]
            imag_val = row_cells[imcol]
            # Handle empty cells (nothing) or non-numeric values as 0.0
            re = isa(real_val, Number) ? Float64(real_val) : 0.0
            im = isa(imag_val, Number) ? Float64(imag_val) : 0.0
            mats[name][k, i, j] = ComplexF64(re, im)
        end
    end
    
    # Create and return final object
    return ImatsReader(String(path), psi_vals, m_vals, nψ, mpert,
               mats[:A], mats[:B], mats[:C], mats[:D], mats[:E], mats[:H])
end

# ────────────────────────────────────────────────────────────────
# Internal: ψ index interpretation
function _ψindex(r::ImatsReader; ψ_index::Union{Nothing,Int}=nothing,
                                   ψ_value::Union{Nothing,Real}=nothing)
    ψ_index !== nothing && return ψ_index
    ψ_value !== nothing || error("Either ψ_index or ψ_value must be specified")
    return argmin(abs.(r.psi .- ψ_value))
end

# ────────────────────────────────────────────────────────────────
"Return single matrix: name ∈ (:A,:B,:C,:D,:E,:H)"
function get_matrix(r::ImatsReader, name::Symbol;
                    ψ_index::Union{Nothing,Int}=nothing,
                    ψ_value::Union{Nothing,Real}=nothing)
    name = Symbol(uppercase(String(name)))
    hasproperty(r, name) || error("unknown matrix $name")
    k = _ψindex(r; ψ_index, ψ_value)
    return getfield(r, name)[k, :, :]
end

"Return one ψ cross-section (6 matrices) as Dict"
function get_matrices(r::ImatsReader; ψ_index::Union{Nothing,Int}=nothing,
                                         ψ_value::Union{Nothing,Real}=nothing)
    k = _ψindex(r; ψ_index, ψ_value)
    return Dict(name => getfield(r, name)[k, :, :] for name in (:A,:B,:C,:D,:E,:H))
end

get_matrices

In [14]:
reader = ImatsReader("../src/DCON/validate/imats.xlsx")


ImatsReader("../src/DCON/validate/imats.xlsx", [0.00999999978, 0.010147579, 0.0105902264, 0.0113276765, 0.0123594841, 0.0136850281, 0.0153035102, 0.0172139555, 0.0194152128, 0.0219059568  …  0.978094041, 0.9805848, 0.982786059, 0.984696507, 0.986314952, 0.9876405, 0.988672316, 0.989409745, 0.989852428, 0.99000001], [-4, -3, -2, -1, 0, 1, 2, 3, 4], 129, 9, ComplexF64[1530.34448 + 0.0im 79.5266571 - 0.000821771799im … -0.00020446493 - 6.27211921e-5im -0.00156646105 - 9.75606526e-5im; 1530.38721 + 0.0im 80.1111069 - 0.0007709497im … -0.000233494109 - 7.31004475e-5im -0.00164888194 - 9.55863288e-5im; … ; 1268.09229 + 0.0im 781.701172 - 0.0895109996im … -3.04324365 + 0.0261531975im 1.90979898 - 0.0214027651im; 1267.49536 + 0.0im 782.051941 - 0.0595153533im … -3.0630877 + 0.0245060399im 1.9160217 - 0.00606394792im;;; 79.5266571 + 0.000821771799im 861.666626 + 0.0im … -0.000896689133 - 0.000150625856im -0.000570992765 + 3.6501915e-5im; 80.1111069 + 0.0007709497im 861.703186 + 0.0im … -0.00093

In [20]:
reshape(matrix_result.amats[129, :], 9, 9)./reader.A[end,:,:]

9×9 Matrix{ComplexF64}:
  1.00036+0.0im          0.999706-8.99822e-5im   …  0.991719-0.00389162im
 0.999706+8.99822e-5im    1.00001+0.0im             0.993011-0.0079452im
 0.997816+0.000185811im  0.999381+9.43505e-5im        1.0015-0.00472755im
 0.993191+0.000269936im  0.997599+0.000190814im      1.00222-0.00292748im
 0.993759+0.00130284im   0.991106+0.000332411im      1.00016-0.00040666im
  1.00115+0.00128139im   0.994771+0.00112444im   …   1.00262+0.000233608im
  1.00364+0.00254335im    1.00133+0.00201552im      0.997848-5.48575e-5im
 0.993661+0.00812152im    1.00241+0.00378235im      0.999171-0.000125075im
 0.991719+0.00389162im   0.993011+0.0079452im        1.00182+0.0im